<a href="https://colab.research.google.com/github/Kevin2558/Data_Science_Borrador/blob/main/04_Modelos_Supervisados/Regresion/SuppVectRegre(SVR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Support Vector Regressor (SVR)

Extensión del modelo Support Vector Machines (SVM) para tareas de regresión. A diferencia de los métodos tradicionales que minimizan directamente el error, SVR buscar encontrar una función que se desvíe lo menos posible de los datos dentro de un margen de tolerancia, penalizando únicamente los errores que exceden ese margen.

Este enfoque permite construir modelos robustos frente a valores atípicos y con buen poder de generalización. SVR puede adaptarse a relaciones lineales y no lineales mediante el uso de funciones *kernel*, lo que lo convierte en una herrameinto flexible para problemas complejos de regresión.

Carguemos las librerías necesarias para la realización del modelo.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, RobustScaler
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

Carguemos el dataset a utilizar.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harlfoxem/housesalesprediction")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/housesalesprediction


In [ ]:
df = pd.read_csv(path + "/kc_house_data.csv")

Separemos las variables entre numéricas y categóricas.

In [ ]:
num = ["sqft_living", "bathrooms", "lat", "long", "sqft_lot", "bedrooms", "yr_built"]
cat = ["zipcode", "waterfront", "condition", "grade"]

In [ ]:
X = df[num + cat]
y = df["price"]

Separemos los datos entre entrenamiento y validación.

In [ ]:
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,random_state=42)

Preprocesamiento.

In [ ]:
prep = ColumnTransformer([
    ("num", RobustScaler(), num),
    ("cat", OneHotEncoder(), cat)
])

Carguemos la librería del modelo.

In [ ]:
from sklearn.svm import SVR

Construyamos el pipeline del modelo SVR.

In [ ]:
svr = Pipeline([
    ("prep", prep),
    ("model", SVR(
        kernel="poly",
        C=10, # Parametro de regularizacion
        epsilon=0.01, # Tolerancia,
        gamma="scale" # Parametro de kernel
    ))
])

Métricas para la evaluación de los modelos.

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

Realicemos el entrenamiento del modelo, la predicción y la evaluación del mismo, a través de las métricas ya definidas.

In [ ]:
svr.fit(X_tr,y_tr)
pred_svr = svr.predict(X_te)
print("RMSE:", rmse(y_te, pred_svr))
print("r2_score:", r2_score(y_te, pred_svr))

RMSE: 403822.0885114913
r2_score: -0.07868650359795493


Notemos que en este caso no funcionó.

Probemos con otros hiperparámetros.

In [ ]:
svr = Pipeline([
    ("prep", prep),
    ("model", SVR(
        kernel="rbf",
        C=100, # Parametro de regularizacion
        epsilon=0.01, # Tolerancia,
        gamma="scale" # Parametro de kernel
    ))
])

svr.fit(X_tr,y_tr)
pred_svr = svr.predict(X_te)
print("RMSE:", rmse(y_te, pred_svr))
print("r2_score:", r2_score(y_te, pred_svr))

RMSE: 382808.7319658623
r2_score: 0.03065410616730102


Notemos que la predicción en este caso es realmente mala, esto se puede deber a la no separabilidad de las variables objetivos.